In [206]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time


symbol = "XRP-USDT"
timeframe = "4hour"
market_type = "spot"
start_time = "2019-01-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# end_time = "2021-06-10 00:00:00"

skip_fetch = False


In [207]:

if not skip_fetch:
    print("Fetching data...")
    def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        time.sleep(0.2)  # Rate limiting
        base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
        url = base_url + "/api/v1/market/candles"
        params = {"type": timeframe, "symbol": symbol.upper()}
        if start_time:
            params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        if end_time:
            params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if data.get("code") == "200000":
                return data["data"]
            else:
                raise Exception(f"KuCoin API error: {data}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request failed: {str(e)}")

    def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        chunks = []
        current_end = end_time
        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        
        while True:
            try:
                chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
                if not chunk:
                    print("No more data available")
                    break
                earliest_ts = int(chunk[-1][0])
                print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
                chunks.extend(chunk)
                if earliest_ts <= start_timestamp:
                    print("Reached start time")
                    break
                current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
        
        if not chunks:
            return []
        
        chunks.sort(key=lambda x: x[0])  # Sort by timestamp
        result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
        print(f"Total candles fetched: {len(result)}")
        return result


    data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
    print(f"Final data length: {len(data)}")
    data
else:
    print("Skipping fetch...")

Fetching data...
Fetched 1500 candles from 2024-04-21 17:00:00
Fetched 1500 candles from 2023-08-15 17:00:00
Fetched 1500 candles from 2022-12-08 16:00:00
Fetched 1500 candles from 2022-04-02 17:00:00
Fetched 1500 candles from 2021-07-26 17:00:00
Fetched 1500 candles from 2020-11-18 16:00:00
Fetched 1500 candles from 2020-03-13 16:00:00
Fetched 1500 candles from 2019-07-07 17:00:00
Fetched 938 candles from 2019-01-01 04:00:00
No more data available
Total candles fetched: 12938
Final data length: 12938


In [208]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df.to_pickle('eth_usdt_4hour.pkl')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12938 entries, 2019-01-01 04:00:00+00:00 to 2024-12-27 12:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    12938 non-null  float64
 1   close   12938 non-null  float64
 2   high    12938 non-null  float64
 3   low     12938 non-null  float64
dtypes: float64(4)
memory usage: 505.4 KB


## creating indicator to set signals long ONLY for simplicity

In [209]:
skip_cell = False

if not skip_cell:
    print("This cell is executed.")

    def ma_cross_rsi_long_only(close_price, ma1_fast, ma2_slow, rsi_buy,rsi_window):
        SMA = vbt.IndicatorFactory.from_talib('SMA')
        ma1 = SMA.run(close_price, timeperiod=ma1_fast).real.to_numpy()
        ma2 = SMA.run(close_price, timeperiod=ma2_slow).real.to_numpy()

        RSI = vbt.IndicatorFactory.from_talib('RSI')
        rsi = RSI.run(close_price, timeperiod=rsi_window).real.to_numpy()
        

        # bb_buy = close_price > upper_band
        rsi_buy_signal = rsi > rsi_buy
        signal_buy = (close_price > ma1) & rsi_buy_signal & (ma1 > ma2) #& bb_buy
        caution_long = (close_price < ma1) & (ma1 > ma2)
        # Check if ma1 and ma2 are too close to each other (percentage difference)
        percentage_diff = np.abs(ma1 - ma2) / ma2
        undecided = percentage_diff < 0.02
        # Define conditions and corresponding values for signals
        conditions = [
            signal_buy & ~undecided,
            caution_long & ~undecided,
            undecided
        ]

        values = [
            100,  # Strong buy signal (close price above short trend MA and RSI above the buy threshold)
            50,   # Caution long
            0     # Undecided
        ]

        # Use np.select to apply the conditions and assign the corresponding values
        signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

        return signal

    # Create the indicator factory
    ma_cross_slow_long_only = vbt.IndicatorFactory(
        class_name='MA_Cross_rsi',
        short_name='ma_rsi',
        input_names=['Close'],
        param_names=['ma1_fast', 'ma2_slow', 'rsi_buy', 'rsi_window'],
        output_names=['ma_rsi']
    ).from_apply_func(ma_cross_rsi_long_only, ma1_fast=20, ma2_slow=50, rsi_buy=70, rsi_window=14)

    # # Define window sizes and RSI thresholds using np.arange
    ma_trend_fast = np.arange(50, 81, 10)  # Range from 20 to 50 with step 5
    ma_trend_slow = np.arange(100, 251, 20)  # Range from 50 to 100 with step 10
    rsi_buy = np.arange(50, 61, 5)  # Range from 60 to 80 with step 5
    rsi_window = np.arange(5, 22, 2)  # Range from 10 to 20 with step 2
    # ma_trend_fast = 50
    # ma_trend_slow = 400
    # rsi_buy = 70
    # rsi_window = [14,7]

    # Run the indicator
    signal = ma_cross_slow_long_only.run(
        df['close'],
        ma1_fast=ma_trend_fast,
        ma2_slow=ma_trend_slow,
        rsi_buy=rsi_buy,
        rsi_window=rsi_window,
        param_product=True  
    )

    signal.ma_rsi

else:
    print("This cell is skipped.")

This cell is executed.


In [210]:
signal.ma_rsi

ma_rsi_ma1_fast            50                             ...  80              \
ma_rsi_ma2_slow           100                             ... 240               
ma_rsi_rsi_buy             50                         55  ...  55  60           
ma_rsi_rsi_window          5  7  9  11 13 15 17 19 21 5   ...  21  5   7   9    
timestamp                                                 ...                   
2019-01-01 04:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   
2019-01-01 08:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   
2019-01-02 12:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   
2019-01-02 20:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   
2019-01-03 00:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   
...                        .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..   
2024-12-26 20:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...  50  50  50  50   
2024-12-27 00:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...  50  50  50  50   
2024-12-27 04:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...  50  50  50  50   
2024-12-27 08:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...  50  50  50  50   
2024-12-27 12:00:00+00:00   0  0  0  0  0  0  0  0  0  0  ...  50  50  50  50   

ma_rsi_ma1_fast                                    
ma_rsi_ma2_slow                                    
ma_rsi_rsi_buy                                     
ma_rsi_rsi_window          11  13  15  17  19  21  
timestamp                                          
2019-01-01 04:00:00+00:00   0   0   0   0   0   0  
2019-01-01 08:00:00+00:00   0   0   0   0   0   0  
2019-01-02 12:00:00+00:00   0   0   0   0   0   0  
2019-01-02 20:00:00+00:00   0   0   0   0   0   0  
2019-01-03 00:00:00+00:00   0   0   0   0   0   0  
...                        ..  ..  ..  ..  ..  ..  
2024-12-26 20:00:00+00:00  50  50  50  50  50  50  
2024-12-27 00:00:00+00:00  50  50  50  50  50  50  
2024-12-27 04:00:00+00:00  50  50  50  50  50  50  
2024-12-27 08:00:00+00:00  50  50  50  50  50  50  
2024-12-27 12:00:00+00:00  50  50  50  50  50  50  

[12938 rows x 864 columns]

In [211]:
def repeat_series_horizontally(series, target_shape):
    data = series.values.reshape(-1, 1)  # Convert to column vector
    return np.repeat(data, target_shape[1], axis=1)

nedded_copies = len(signal.ma_rsi.columns)
print('needed columns: ',nedded_copies)
print('needed rows: ',len(signal.ma_rsi))
print('shape:', signal.ma_rsi.shape)
print()

# copying the close price to the shape of the indicator
print('original shape:\n\n', df['close'])
dublicate_close = repeat_series_horizontally(df['close'], signal.ma_rsi.shape)
df_dublicate_close  = pd.DataFrame(dublicate_close,df.index)
df_dublicate_close 



needed columns:  864
needed rows:  12938
shape: (12938, 864)

original shape:

 timestamp
2019-01-01 04:00:00+00:00    0.355000
2019-01-01 08:00:00+00:00    0.353961
2019-01-02 12:00:00+00:00    0.360906
2019-01-02 20:00:00+00:00    0.369254
2019-01-03 00:00:00+00:00    0.363271
                               ...   
2024-12-26 20:00:00+00:00    2.155950
2024-12-27 00:00:00+00:00    2.175020
2024-12-27 04:00:00+00:00    2.143690
2024-12-27 08:00:00+00:00    2.214070
2024-12-27 12:00:00+00:00    2.206280
Name: close, Length: 12938, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,854,855,856,857,858,859,860,861,862,863
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-01-01 04:00:00+00:00,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,...,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000,0.355000
2019-01-01 08:00:00+00:00,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,...,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961,0.353961
2019-01-02 12:00:00+00:00,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,...,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906,0.360906
2019-01-02 20:00:00+00:00,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,...,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254,0.369254
2019-01-03 00:00:00+00:00,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,...,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271,0.363271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-26 20:00:00+00:00,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,...,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950,2.155950
2024-12-27 00:00:00+00:00,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,...,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020,2.175020
2024-12-27 04:00:00+00:00,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,...,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690,2.143690


## compare the shape of the copied closing prices 

In [212]:
df_custom_indiator = signal.ma_rsi

if df_custom_indiator.shape == df_dublicate_close.shape:
    df_renamed_close = df_dublicate_close
    df_renamed_close.columns = df_custom_indiator.columns
    print("Columns have been renamed ")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_renamed_close.head()
df_renamed_close

Columns have been renamed 


ma_rsi_ma1_fast                  50                                          \
ma_rsi_ma2_slow                 100                                           
ma_rsi_rsi_buy                   50                                           
ma_rsi_rsi_window                5         7         9         11        13   
timestamp                                                                     
2019-01-01 04:00:00+00:00  0.355000  0.355000  0.355000  0.355000  0.355000   
2019-01-01 08:00:00+00:00  0.353961  0.353961  0.353961  0.353961  0.353961   
2019-01-02 12:00:00+00:00  0.360906  0.360906  0.360906  0.360906  0.360906   
2019-01-02 20:00:00+00:00  0.369254  0.369254  0.369254  0.369254  0.369254   
2019-01-03 00:00:00+00:00  0.363271  0.363271  0.363271  0.363271  0.363271   
...                             ...       ...       ...       ...       ...   
2024-12-26 20:00:00+00:00  2.155950  2.155950  2.155950  2.155950  2.155950   
2024-12-27 00:00:00+00:00  2.175020  2.175020  2.175020  2.175020  2.175020   
2024-12-27 04:00:00+00:00  2.143690  2.143690  2.143690  2.143690  2.143690   
2024-12-27 08:00:00+00:00  2.214070  2.214070  2.214070  2.214070  2.214070   
2024-12-27 12:00:00+00:00  2.206280  2.206280  2.206280  2.206280  2.206280   

ma_rsi_ma1_fast                                                              \
ma_rsi_ma2_slow                                                               
ma_rsi_rsi_buy                                                           55   
ma_rsi_rsi_window                15        17        19        21        5    
timestamp                                                                     
2019-01-01 04:00:00+00:00  0.355000  0.355000  0.355000  0.355000  0.355000   
2019-01-01 08:00:00+00:00  0.353961  0.353961  0.353961  0.353961  0.353961   
2019-01-02 12:00:00+00:00  0.360906  0.360906  0.360906  0.360906  0.360906   
2019-01-02 20:00:00+00:00  0.369254  0.369254  0.369254  0.369254  0.369254   
2019-01-03 00:00:00+00:00  0.363271  0.363271  0.363271  0.363271  0.363271   
...                             ...       ...       ...       ...       ...   
2024-12-26 20:00:00+00:00  2.155950  2.155950  2.155950  2.155950  2.155950   
2024-12-27 00:00:00+00:00  2.175020  2.175020  2.175020  2.175020  2.175020   
2024-12-27 04:00:00+00:00  2.143690  2.143690  2.143690  2.143690  2.143690   
2024-12-27 08:00:00+00:00  2.214070  2.214070  2.214070  2.214070  2.214070   
2024-12-27 12:00:00+00:00  2.206280  2.206280  2.206280  2.206280  2.206280   

ma_rsi_ma1_fast            ...        80                                \
ma_rsi_ma2_slow            ...       240                                 
ma_rsi_rsi_buy             ...        55        60                       
ma_rsi_rsi_window          ...        21        5         7         9    
timestamp                  ...                                           
2019-01-01 04:00:00+00:00  ...  0.355000  0.355000  0.355000  0.355000   
2019-01-01 08:00:00+00:00  ...  0.353961  0.353961  0.353961  0.353961   
2019-01-02 12:00:00+00:00  ...  0.360906  0.360906  0.360906  0.360906   
2019-01-02 20:00:00+00:00  ...  0.369254  0.369254  0.369254  0.369254   
2019-01-03 00:00:00+00:00  ...  0.363271  0.363271  0.363271  0.363271   
...                        ...       ...       ...       ...       ...   
2024-12-26 20:00:00+00:00  ...  2.155950  2.155950  2.155950  2.155950   
2024-12-27 00:00:00+00:00  ...  2.175020  2.175020  2.175020  2.175020   
2024-12-27 04:00:00+00:00  ...  2.143690  2.143690  2.143690  2.143690   
2024-12-27 08:00:00+00:00  ...  2.214070  2.214070  2.214070  2.214070   
2024-12-27 12:00:00+00:00  ...  2.206280  2.206280  2.206280  2.206280   

ma_rsi_ma1_fast                                                              \
ma_rsi_ma2_slow                                                               
ma_rsi_rsi_buy                                                                
ma_rsi_rsi_window                11        13   

In [223]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if c.close[c.i-1,c.col] <= sl_prices[c.i]:
        # if low[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                slippage=0.01)
            # print('sl hit at index',c.i)
            # print('sl order', value)
            return value

        # sl update for long position

        if c.i >= 5:  # Ensure there are enough bars to look back
            lowest_low = np.min(low[c.i-3:c.i])  # Find the lowest low in the last 5 bars
            if (entries[c.i, c.col] == 75):
                # print('sl update before',c.i,sl_prices[c.i])
                update1 = lowest_low * 0.90
                if update1 > sl_prices[c.i]:
                    sl_prices[:]= update1
                    # print('sl update after at index',c.i,sl_prices[c.i])
            if (entries[c.i, c.col] == 50):  
                update2 = lowest_low * 0.9
                if update2 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update2       
            if (entries[c.i, c.col] == 0):  
                update3 = close_price * 0.98
                if update3 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update3      



        # if c.position_now > 0:
        #     if (high[c.i-1,c.col] >= tp_prices[c.i])&(tp_prices[c.i] != np.nan):
        #         value = vbt.portfolio.nb.order_nb(
        #             size=-c.position_now/2,  # Close position
        #             price=tp_prices[c.i],
        #             size_type=SizeType.Amount,
        #             direction=Direction.LongOnly)
        #         tp_prices[:] = np.nan
        #         sl_prices[:] = entry_price[c.i]
        #         # print('tp hit at index',c.i)
        #         return value
                


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i, c.col] == 100:
            entry_price[:] = close_price
            sl_price = close_price * 0.90
            tp_prices[:] = close_price * 1.05
            order = vbt.portfolio.nb.order_nb(
                size=0.3,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.01,  # No fees
                slippage=0.01,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )
            lowest_low = np.min(low[c.i-5:c.i]) 
            sl_prices[:] = entry_price[c.i] * 0.90
            # print('order', order)
            return order

    
    return vbt.portfolio.enums.NoOrder


# open_ =repeat_series_horizontally(df['open'],signal.ma_rsi.shape)
# print(type(open_))
# high = repeat_series_horizontally(df['high'],signal.ma_rsi.shape)
# low = repeat_series_horizontally(df['low'],signal.ma_rsi.shape)
close = df_renamed_close
entries = df_custom_indiator.to_numpy()

open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()



# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array
# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,  # Pass the SL prices array
    init_cash=500  # Initial cash balance
)

# # Display some portfolio performance metrics
# print("Total Return:", pf.total_return())
# print("\nOrder Records:")

# pf.orders.records_readable

# Optional: Plot equity curve
# pf.plot().show()



In [224]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(20)

total_return  \
ma_rsi_ma1_fast ma_rsi_ma2_slow ma_rsi_rsi_buy ma_rsi_rsi_window                 
70              100             60             19                     1.773763   
                                50             15                     1.758607   
80              200             60             19                     1.743113   
70              100             60             21                     1.740828   
80              200             60             21                     1.722828   
70              200             60             19                     1.602146   
                                               21                     1.596263   
                100             50             21                     1.577408   
80              100             60             19                     1.568555   
70              100             50             13                     1.564187   
                200             50             15                     1.528249   
50              100             60             19                     1.504685   
70              100             60             17                     1.501074   
80              240             60             21                     1.490649   
                                               19                     1.484733   
50              180             50             15                     1.475980   
                100             60             21                     1.458596   
                                50             15                     1.440195   
                160             50             15                     1.434898   
                220             60             19                     1.434755   

                                                                  max_drawdown  
ma_rsi_ma1_fast ma_rsi_ma2_slow ma_rsi_rsi_buy ma_rsi_rsi_window                
70              100             60             19                    -0.208884  
                                50             15                    -0.284416  
80              200             60             19                    -0.258834  
70              100             60             21                    -0.192973  
80              200             60             21                    -0.246629  
70              200             60             19                    -0.261723  
                                               21                    -0.249341  
                100             50             21                    -0.302605  
80              100             60             19                    -0.142018  
70              100             50             13                    -0.274602  
                200             50             15                    -0.367559  
50              100             60             19                    -0.273157  
70              100             60             17                    -0.222456  
80              240             60             21                    -0.204741  
                                               19                    -0.234717  
50              180             50             15                    -0.371530  
                100             60             21                    -0.273015  
                                50             15                    -0.340769  
                160             50             15                    -0.399924  
                220             60             19                    -0.355748

In [215]:

df_sorted = pf.total_return()
df_sorted.vbt.volume(x_level='ma_rsi_ma1_fast', y_level='ma_rsi_ma2_slow', z_level='ma_rsi_rsi_buy')


FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hovertemplate': ('ma_rsi_ma1_fast: %{x}<br>ma_rs' ... 'value: %{value}<extra></extra>'),
              'opacity': 0.2,
              'surface': {'count': 15},
              'type': 'volume',
              'uid': 'bca467c8-4316-40c0-8757-3f61de418b78',
              'value': array([0.42450143, 0.30821049, 0.40243383, 0.40337119, 0.2957626 , 0.33515433,
                              0.39144575, 0.31287651, 0.33835662, 0.39240345, 0.32032848, 0.35665674,
                              0.3694187

In [216]:

df_sorted = pf.total_return()
df_sorted.vbt.heatmap(x_level='ma_rsi_ma1_fast', y_level='ma_rsi_ma2_slow', slider_level='ma_rsi_rsi_buy')

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'ma_rsi_ma1_fast: %{x}<br>ma_rsi_ma2_slow: %{y}<br>value: %{z}<extra></extra>',
              'name': '50',
              'type': 'heatmap',
              'uid': 'e4fb986c-e4b9-4451-a442-7413d503a612',
              'visible': True,
              'x': array([50, 60, 70, 80]),
              'y': array([100, 120, 140, 160, 180, 200, 220, 240]),
              'z': array([[0.42450143, 0.33726553, 0.44221312, 0.30715485],
                      